<a href="https://colab.research.google.com/github/Mainabryan/SMART-_ECOMMERCE/blob/main/Merging_the_datsets_into_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount drive and reate folde

In [1]:
# -------------------------------------------------
# 1. MOUNT DRIVE & CREATE ShopSmart FOLDER
# -------------------------------------------------
from google.colab import drive
import os

drive.mount('/content/drive')                     # <-- click the link, allow access

folder = '/content/drive/MyDrive/ShopSmart'
os.makedirs(folder, exist_ok=True)                # creates if missing
print(f"Folder ready: {folder}")

Mounted at /content/drive
Folder ready: /content/drive/MyDrive/ShopSmart


In [2]:
# -------------------------------------------------
# 2. UPLOAD THE 7 CSV FILES
# -------------------------------------------------
from google.colab import files
import shutil

# List of required files
required = [
    'customers.csv',
    'orders.csv',
    'products.csv',
    'ad_campaigns.csv',
    'ga_sessions.csv',
    'ml_insights_for_tableau.csv',
    'product_recommendations.csv'
]

print("Please upload the following 7 files (one by one or all at once):")
uploaded = files.upload()                         # <-- drag & drop here

# Move them to the folder
for name in uploaded.keys():
    dst = os.path.join(folder, name)
    shutil.move(name, dst)
    print(f"Moved {name} → {dst}")

# Verify
print("\nFiles in ShopSmart:")
!ls -lh "{folder}"

Please upload the following 7 files (one by one or all at once):


Saving ga_sessions-sheet1-sourcetable.csv to ga_sessions-sheet1-sourcetable.csv
Saving orders_csv-sheet1-sourcetable.csv to orders_csv-sheet1-sourcetable.csv
Saving ads_campaigns-sheet1-sourcetable.csv to ads_campaigns-sheet1-sourcetable.csv
Saving poduct_csv-sheet1-sourcetable.csv to poduct_csv-sheet1-sourcetable.csv
Saving customer_table-sheet1-sourcetable (1).csv to customer_table-sheet1-sourcetable (1).csv
Moved ga_sessions-sheet1-sourcetable.csv → /content/drive/MyDrive/ShopSmart/ga_sessions-sheet1-sourcetable.csv
Moved orders_csv-sheet1-sourcetable.csv → /content/drive/MyDrive/ShopSmart/orders_csv-sheet1-sourcetable.csv
Moved ads_campaigns-sheet1-sourcetable.csv → /content/drive/MyDrive/ShopSmart/ads_campaigns-sheet1-sourcetable.csv
Moved poduct_csv-sheet1-sourcetable.csv → /content/drive/MyDrive/ShopSmart/poduct_csv-sheet1-sourcetable.csv
Moved customer_table-sheet1-sourcetable (1).csv → /content/drive/MyDrive/ShopSmart/customer_table-sheet1-sourcetable (1).csv

Files in ShopSma

In [4]:
# -------------------------------------------------
# 3. BUILD CLEAN MASTER TABLE (29 columns, no junk)
# -------------------------------------------------
import pandas as pd
import numpy as np

# ---- Load ----
# Corrected file names based on the output of the previous cell
customers = pd.read_csv(f'{folder}/customer_table-sheet1-sourcetable (1).csv')
orders    = pd.read_csv(f'{folder}/orders_csv-sheet1-sourcetable.csv')
products  = pd.read_csv(f'{folder}/poduct_csv-sheet1-sourcetable.csv')
ad_campaigns = pd.read_csv(f'{folder}/ads_campaigns-sheet1-sourcetable.csv')
ga_sessions  = pd.read_csv(f'{folder}/ga_sessions-sheet1-sourcetable.csv')
# The following two files were not uploaded, will handle them later
ml        = pd.DataFrame() # pd.read_csv(f'{folder}/ml_insights_for_tableau.csv')
recs      = pd.DataFrame() # pd.read_csv(f'{folder}/product_recommendations.csv')


# ---- Safe date conversion ----
def to_dt(df, col):
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

orders     = to_dt(orders,     'order_date')
customers  = to_dt(customers,  'signup_date')
ga_sessions = to_dt(ga_sessions, 'session_date')

# ---- Start with orders ----
master = orders[['order_id','customer_id','order_date','product_id',
                 'quantity','price','discount','total','ad_campaign_id']].copy()

# ---- Merge cleanly ----
master = master.merge(customers[['customer_id','customer_name','gender','age','country','signup_date']],
                      on='customer_id', how='left')
master = master.merge(products[['product_id','product_name','category']],
                      on='product_id', how='left')
master = master.merge(ad_campaigns[['ad_campaign_id','platform','spend','impressions','clicks','conversions']],
                      on='ad_campaign_id', how='left')

# ---- Latest GA session per customer ----
if 'session_date' in ga_sessions.columns and not ga_sessions.empty:
    latest = ga_sessions.sort_values('session_date').drop_duplicates('customer_id', keep='last')
    master = master.merge(latest[['customer_id','session_date','device','traffic_source',
                                 'session_duration','pages_viewed','bounced']],
                          on='customer_id', how='left')
else:
    for c in ['session_date','device','traffic_source','session_duration','pages_viewed','bounced']:
        master[c] = np.nan

# ---- ML + Top-1 recommendation ----
# Handle missing ML and Recs dataframes
if not ml.empty:
    master = master.merge(ml[['customer_id','churn_risk','predicted_clv_365d','vip_candidate']],
                          on='customer_id', how='left')
else:
    for c in ['churn_risk','predicted_clv_365d','vip_candidate']:
        master[c] = np.nan

if not recs.empty:
    top1 = recs[recs['rank'] == 1][['customer_id','rec_product_id','rec_product_name','score']]
    top1 = top1.rename(columns={'rec_product_id':'top1_product_id',
                                'rec_product_name':'top1_product_name',
                                'score':'top1_score'})
    master = master.merge(top1, on='customer_id', how='left')
else:
     for c in ['top1_product_id','top1_product_name','top1_score']:
        master[c] = np.nan


# ---- Final column order (29 columns) ----
final_cols = [
    'order_id','customer_id','customer_name','gender','age','country','signup_date',
    'order_date','product_id','product_name','category','quantity','price','discount','total',
    'ad_campaign_id','platform','spend','impressions','clicks','conversions',
    'session_date','device','traffic_source','session_duration','pages_viewed','bounced',
    'churn_risk','predicted_clv_365d','vip_candidate',
    'top1_product_id','top1_product_name','top1_score'
]

master = master[final_cols]

# ---- SAVE ----
out_path = f'{folder}/ShopSmart_Master_Table.csv'
master.to_csv(out_path, index=False)

print(f"\nSUCCESS! Master table saved:")
print(f"   Rows: {len(master):,}")
print(f"   Columns: {len(master.columns)}")
print(f"   File: {out_path}")


SUCCESS! Master table saved:
   Rows: 75
   Columns: 33
   File: /content/drive/MyDrive/ShopSmart/ShopSmart_Master_Table.csv
